# FINM320 Numerical Method
# HW2
# Hannah Wang

# Question 1

In [1]:
import numpy as np

In [2]:
class localvolDynamics:
    
    def __init__(self, S0, r, q, maxvol, localvol): 
        self.S0 = S0
        self.r = r
        self.q = q
        self.maxvol = maxvol
        self.localvol = localvol


In [3]:
hw2dynamics = localvolDynamics(S0 = 100, r = 0.06, q = 0.01, maxvol = 0.6, 
                     localvol = lambda S,t: np.minimum(0.2+5*np.log(S/100)**2+0.1*np.exp(-t), 0.6))

# Note that hw2dynamics.localvol is a function 
# that may be invoked in the usual way, for example:
# hw2dynamics.localvol( exchangerate , time )

In [4]:
class CallOnAmericanPut:

    def __init__(self, putexpiry, putstrike, callexpiry, callstrike):
        self.putexpiry = putexpiry
        self.putstrike = putstrike
        self.callexpiry = callexpiry
        self.callstrike = callstrike
        

In [5]:
hw2contract = CallOnAmericanPut(putexpiry=0.75, putstrike=95, callexpiry=0.25, callstrike=10)

In [12]:
class TrinomTree:
    
    def __init__(self, N):
        self.N = N
        
    # You complete the coding of this function
    def price_compound_localvol(self, contract, dynamics):
        
        put_expiry, put_strike = contract.putexpiry, contract.putstrike
        call_expiry, call_strike = contract.callexpiry, contract.callstrike
        S0, r, q = dynamics.S0, dynamics.r, dynamics.q
        max_vol, local_vol = dynamics.maxvol, dynamics.localvol
        
        delta_t = put_expiry / self.N
        representative_vol = local_vol(S0, 0)
        delta_x = max(representative_vol * np.sqrt(3 * delta_t), max_vol * np.sqrt(delta_t))
        S = S0 * np.exp(np.linspace(self.N, -self.N, num=2*self.N+1, endpoint=True) * delta_x)
        put_price = np.maximum(put_strike - S, 0)
        
        
        for t in np.linspace(self.N-1, 0, num=self.N, endpoint=True) * delta_t:
            S = S[1:-1]
            sigma = local_vol(S, t)
            
            nu = (r - q) - sigma ** 2/2
            P_u = 0.5 * ((sigma ** 2 * delta_t + nu ** 2 * delta_t ** 2) / delta_x ** 2 + nu * delta_t / delta_x)
            P_d = 0.5 * ((sigma ** 2 * delta_t + nu ** 2 * delta_t ** 2) / delta_x ** 2 - nu * delta_t / delta_x)
            P_m = 1 - (sigma ** 2 * delta_t + nu ** 2 * delta_t ** 2) / delta_x ** 2
            
            up_put_prices = put_price[:-2]
            mid_put_prices = put_price[1:-1]
            down_put_prices = put_price[2:]
            
            put_intrinsic_value = np.maximum(put_strike - S, 0)
            put_price = np.maximum(put_intrinsic_value,
                                   np.exp(-r * delta_t) * (P_u * up_put_prices + P_m * mid_put_prices + P_d * down_put_prices))
            
            if abs(t-call_expiry)<0.0001:
                call_price = np.maximum(put_price - call_strike, 0)
            elif t < call_expiry:
                up_call_prices = call_price[:-2]
                mid_call_prices = call_price[1:-1]
                down_call_prices = call_price[2:]
                call_price = np.exp(-r * delta_t) * (P_u * up_call_prices + P_m * mid_call_prices + P_d * down_call_prices)
                
        price_of_put = put_price[0]
        
        price_of_call_on_put = call_price[0]
        
        return (price_of_put, price_of_call_on_put)

    

In [49]:
hw2tree = TrinomTree(N=9000)  #change if necessary to get $0.01 accuracy, in your judgment

In [50]:
(answer_part_a, answer_part_b) = hw2tree.price_compound_localvol(hw2contract,hw2dynamics)

In [51]:
(answer_part_a, answer_part_b) 

(7.007311562145161, 1.591881397156099)

# Question 2

## 2(a)

The exact formula for the Black-Scholes delta of a vanilla call is

\begin{equation}
\Delta = N(d_1),
\end{equation}

where

\begin{equation}
d_1 = \frac{\log(S / K) + (r + \frac{\sigma^2}{2}) (T - t)}{\sigma \sqrt{T - t}} = \frac{(r + \frac{\sigma^2}{2}) \sqrt{T}}{\sigma}.
\end{equation}

if $S = K$ and $t = 0$.


By first-order Taylor expansion at $x = 0$,

\begin{equation}
N(x) \approx N(0) + N'(0) \cdot x.
\end{equation}

We know that $N(0) = 0.5$ and $N'(x) = \frac{1}{\sqrt{2 \pi}} \exp(\frac{-x^2}{2})$, $N'(0) = \frac{1}{\sqrt{2\pi}} \approx 0.4$. Also, $\sigma = 0.2$, $T = 0.25$, and $r = 0.01$.

Thus,

\begin{align}
\Delta &\approx 0.5 + 0.4 \cdot \frac{(r + \frac{\sigma^2}{2}) \sqrt{T}}{\sigma}\\
&= 0.5 + 0.4 \cdot \frac{(0.01 + \frac{0.2^2}{2}) \sqrt{0.25}}{0.2}\\
&= 0.53.
\end{align}


## 2(b)

Since dollar delta is 3, then $S_0 \Delta\ = 3$, $\Delta\ = 3/4$.

Since dollar gamma is 0.02, then $\frac{1}{100}S_0^2 \Gamma = 0.02$, $\Gamma\ = 1/8$.

By second-order Taylor expansion at $S_0$, $C(S) \approx C(S_0) + \Delta (S - S_0) + \frac{\Gamma}{2} (S - S_0)^2$.

Thus, $C(3.6) \approx 5 + \frac{3}{4} (3.6 - 4) + \frac{1}{16} (3.6 - 4)^2 = 4.71$.
